In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(587, with_units=False)
Hist_table


Year,Entries
IntegerAndSexagesimal,Temporal
365 ;,"10 ; 15,11,23"
730 ;,"21 ; 06,22,47"
1095 ;,"2 ; 08,50,07"
1461 ;,"14 ; 00,01,31"
1826 ;,"24 ; 15,12,54"
2191 ;,"5 ; 17,40,14"
2556 ;,"16 ; 08,51,30"
2922 ;,"29 ; 00,03,01"
3287 ;,"9 ; 02,30,21"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(23):
    Test_recomp[i][1]=Temporal('0;0,0,0')
    Delta_recomp[i][1]=Temporal('0;0,0,0')

In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)


for i in range(23):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=5):
        elong=(daily_mean*Hist_table[i][0])%REVO
        time=elong/daily_mean
        time=Temporal(time,7)
        
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*time
            Delta_recomp[i][1]=24*60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Year,Entries
IntegerAndSexagesimal,Temporal
365 ;,"10 ; 15,11,23"
730 ;,"21 ; 06,22,47"
1095 ;,"2 ; 08,50,07"
1461 ;,"14 ; 00,01,31"
1826 ;,"24 ; 15,12,54"
2191 ;,"5 ; 17,40,14"
2556 ;,"16 ; 08,51,38"
2922 ;,"28 ; 00,03,01"
3287 ;,"9 ; 02,30,21"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(41321.391304347824, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
Year,
365 ;,0
730 ;,0
1095 ;,0
1461 ;,0
1826 ;,0
2191 ;,0
2556 ;,-8
2922 ;,86400
3287 ;,0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy date years expanded.csv')
B.to_csv('Mean syzygy date years expanded.zip', index=False, compression=compression_opts)